# LA to Chicago
The first step in the process generating a route form LA to Chicago is to load the NARN database. From this we can extract only the tracks that are either owned by or have usage rights granted to BNSF. We also need to "starting" and "ending" node. These have been determiend previously as 320,998 and 409,862 respectively.

In [1]:
import geopandas as gpd
import networkx as nx
from shapely import geometry, ops
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd
import folium

In [2]:
data = gpd.read_file("North_American_Rail_Network_Lines.geojson")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


There are a significant number of "lines" in this database.

In [3]:
len(data)

302988

In [9]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 302988 entries, 0 to 302987
Data columns (total 35 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   OBJECTID       302988 non-null  int64   
 1   FRAARCID       302988 non-null  int64   
 2   FRFRANODE      302988 non-null  int64   
 3   TOFRANODE      302988 non-null  int64   
 4   STFIPS         235945 non-null  object  
 5   CNTYFIPS       235945 non-null  object  
 6   STCNTYFIPS     235945 non-null  object  
 7   STATEAB        302988 non-null  object  
 8   COUNTRY        302988 non-null  object  
 9   FRADISTRCT     302988 non-null  int64   
 10  RROWNER1       299857 non-null  object  
 11  RROWNER2       3006 non-null    object  
 12  RROWNER3       27 non-null      object  
 13  TRKRGHTS1      51734 non-null   object  
 14  TRKRGHTS2      13823 non-null   object  
 15  TRKRGHTS3      3252 non-null    object  
 16  TRKRGHTS4      919 non-null     object  
 17  TR

Lets first restict them to the ones that can be used by BNSF.

In [5]:
bnsf_data = data[data[['RROWNER1', 'RROWNER2', 'RROWNER3', 'TRKRGHTS1', 'TRKRGHTS2', 'TRKRGHTS3', 'TRKRGHTS4', 'TRKRGHTS5', 'TRKRGHTS6', 'TRKRGHTS7', 'TRKRGHTS8', 'TRKRGHTS9']].isin(['BNSF']).any(axis=1)]

In [6]:
len(bnsf_data)

42187

In [8]:
start = bnsf_data[bnsf_data['TOFRANODE'].isin([320998])]

In [10]:
start

,OBJECTID,FRAARCID,FRFRANODE,TOFRANODE,STFIPS,CNTYFIPS,STCNTYFIPS,STATEAB,COUNTRY,FRADISTRCT,...,YARDNAME,PASSNGR,STRACNET,TRACKS,NET,MILES,KM,TIMEZONE,ShapeSTLength,geometry
36307,36308,336446,321043,320998,06,037,06037,CA,US,7,...,None,None,None,1.0,M,0.253699,0.408289,P,491.521964,"LINESTRING (-118.24918 33.75628, -118.25063 33..."


In [11]:
start.geometry

36307    LINESTRING (-118.24918 33.75628, -118.25063 33...
Name: geometry, dtype: geometry